## I made it work but it's really slow. Also I need to figure out a way to find the accuracy for this algorithm.

In [1]:
import pandas as pd
import numpy as np
import gensim.downloader as api
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('stopwords')

PATH_TO_DATASET = 'complaints.csv' 

data = pd.read_csv(PATH_TO_DATASET)
data = data.fillna('')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zenan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zenan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\zenan\AppData\Local\Temp\ipykernel_15380\1903783833.py:15: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(PATH_TO_DATASET)


In [2]:
# Load pre-trained word embeddings
word2vec_model = api.load("glove-wiki-gigaword-100")

In [3]:
# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

In [4]:
# Convert the preprocessed text to vectors
def text_to_vector(tokens):
    vector = np.zeros(word2vec_model.vector_size)
    for token in tokens:
        if token in word2vec_model:
            vector += word2vec_model[token]
    return vector / len(tokens)

## testing sample starts here, needs to find the best threshold and determine the accuracy.

In [12]:
x = 1000
df = data.sample(n=x, random_state=42)
df.reset_index(drop=True, inplace=True)

In [13]:
test_df = df.copy()
test_df['combined_text'] = test_df['Product'] + ' ' + test_df['Sub-product'] + ' ' + test_df['Issue'] + ' ' + test_df['Sub-issue'] + ' ' + test_df['Consumer complaint narrative'] + ' ' + test_df['Company public response'] + ' ' + test_df['Company'] + ' ' + test_df['Company response to consumer']

In [14]:
test_df['processed_combined_text'] = test_df['combined_text'].apply(preprocess_text)

In [15]:
test_df['text_vector'] = test_df['processed_combined_text'].apply(text_to_vector)

In [16]:
test_df['text_vector']

0      [0.19270455547504955, -0.1130928890262213, 0.1...
1      [-0.01269579455256462, -0.2328399963211268, 0....
2      [0.3118487522006035, 0.15755712456302717, 0.25...
3      [0.004078068504376071, 0.08383612808193193, 0....
4      [0.05192294147084741, 0.0026056262500145856, 0...
                             ...                        
995    [0.2051051693658034, 0.2898755554213292, 0.176...
996    [-0.019825622730422765, 0.05703169174375944, 0...
997    [0.07744729475063436, -0.07664940993794624, 0....
998    [0.15580810173414647, 0.17453869070159272, 0.1...
999    [0.05888128497425486, 0.14730820582137283, 0.1...
Name: text_vector, Length: 1000, dtype: object

In [17]:
similarity_threshold = 0.999
drop_indices = []

for i in range(len(test_df)):
    if i not in drop_indices:
        vector1 = test_df.at[i, 'text_vector']
        for j in range(i+1, len(test_df)):
            if j not in drop_indices:
                vector2 = test_df.at[j, 'text_vector']
                similarity = cosine_similarity([vector1], [vector2])[0][0]
                if similarity > similarity_threshold:
                    drop_indices.append(j)

In [18]:
drop_indices

[131,
 205,
 446,
 804,
 950,
 28,
 113,
 175,
 345,
 390,
 433,
 452,
 502,
 549,
 595,
 617,
 622,
 640,
 645,
 683,
 721,
 783,
 785,
 795,
 854,
 867,
 903,
 924,
 946,
 987,
 574,
 19,
 25,
 39,
 105,
 134,
 165,
 194,
 197,
 207,
 213,
 238,
 282,
 305,
 350,
 363,
 376,
 383,
 441,
 444,
 462,
 479,
 518,
 547,
 550,
 553,
 567,
 568,
 673,
 681,
 704,
 730,
 791,
 847,
 880,
 931,
 973,
 993,
 53,
 137,
 156,
 223,
 563,
 779,
 868,
 149,
 728,
 52,
 59,
 89,
 136,
 218,
 474,
 615,
 635,
 668,
 731,
 775,
 24,
 167,
 421,
 438,
 657,
 930,
 934,
 315,
 548,
 565,
 748,
 851,
 986,
 69,
 221,
 385,
 509,
 560,
 648,
 418,
 534,
 970,
 513,
 171,
 284,
 494,
 495,
 551,
 908,
 912,
 443,
 472,
 693,
 764,
 925,
 929,
 230,
 248,
 250,
 707,
 980,
 889,
 212,
 286,
 516,
 210,
 266,
 312,
 853,
 575,
 666,
 685,
 262,
 624,
 794,
 873,
 981,
 348,
 465,
 510,
 186,
 397,
 526,
 768,
 226,
 270,
 292,
 293,
 333,
 456,
 488,
 505,
 544,
 583,
 599,
 650,
 733,
 778,
 818,
 834,
 9

In [19]:
test_df.drop(drop_indices, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [20]:
#test_df.drop(columns=['processed_combined_text', 'text_vector'], inplace=True)

In [21]:
test_df

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,...,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,combined_text,processed_combined_text,text_vector
0,2021-04-19,Mortgage,FHA mortgage,Struggling to pay mortgage,,,,"Community Loan Servicing, LLC (formerly known ...",CA,91335,...,Other,Web,2021-04-19,Closed with explanation,Yes,,4309853,Mortgage FHA mortgage Struggling to pay mortga...,"[mortgage, fha, mortgage, struggling, pay, mor...","[0.19270455547504955, -0.1130928890262213, 0.1..."
1,2020-04-29,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,,,"EQUIFAX, INC.",NJ,07075,...,,Phone,2020-04-29,Closed with explanation,Yes,,3628280,"Credit reporting, credit repair services, or o...","[credit, reporting, credit, repair, services, ...","[-0.01269579455256462, -0.2328399963211268, 0...."
2,2015-08-14,Debt collection,Medical,Cont'd attempts collect debt not owed,Debt was paid,,,"Western Mercantile Agency, Inc",OR,97459.0,...,Consent not provided,Web,2015-08-27,Closed with explanation,Yes,No,1520100,Debt collection Medical Cont'd attempts collec...,"[debt, collection, medical, cont, attempts, co...","[0.3118487522006035, 0.15755712456302717, 0.25..."
3,2020-08-18,Credit card or prepaid card,Store credit card,Problem with a purchase shown on your statement,Credit card company isn't resolving a dispute ...,I purchased 4 utility vest from XXXX XXXX on X...,,"Bread Financial Holdings, Inc.",AL,35210,...,Consent provided,Web,2020-08-27,Closed with monetary relief,Yes,,3802506,Credit card or prepaid card Store credit card ...,"[credit, card, prepaid, card, store, credit, c...","[0.004078068504376071, 0.08383612808193193, 0...."
4,2021-02-13,Credit card or prepaid card,General-purpose credit card or charge card,"Other features, terms, or problems",Other problem,,,KEYCORP,NY,14028,...,Consent not provided,Web,2021-03-04,Closed with explanation,Yes,,4138291,Credit card or prepaid card General-purpose cr...,"[credit, card, prepaid, card, general, purpose...","[0.05192294147084741, 0.0026056262500145856, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,2023-01-06,Debt collection,Other debt,Written notification about debt,Didn't receive enough information to verify debt,,Company has responded to the consumer and the ...,"SUNRISE CREDIT SERVICES, INC",PA,17022.0,...,Consent not provided,Web,2023-01-06,Closed with explanation,Yes,,6404248,Debt collection Other debt Written notificatio...,"[debt, collection, debt, written, notification...","[0.11124708324670791, 0.16916799698024987, 0.2..."
761,2014-09-04,Debt collection,I do not know,Cont'd attempts collect debt not owed,Debt is not mine,,,Oracle Financial Group LLC.,FL,33142,...,,Web,2014-09-04,Closed with non-monetary relief,Yes,No,1015346,Debt collection I do not know Cont'd attempts ...,"[debt, collection, know, cont, attempts, colle...","[0.2051051693658034, 0.2898755554213292, 0.176..."
762,2017-07-26,Checking or savings account,Other banking product or service,Problem caused by your funds being low,Overdrafts and overdraft fees,,,ASTORIA BANK,NY,11772,...,,Referral,2017-07-27,Closed with explanation,Yes,,2587115,Checking or savings account Other banking prod...,"[checking, savings, account, banking, product,...","[0.07744729475063436, -0.07664940993794624, 0...."
763,2017-01-23,Student loan,Federal student loan servicing,Can't repay my loan,Can't decrease my monthly payments,Requested to have my school loan income lowere...,,"Navient Solutions, LLC.",CA,95020,...,Consent provided,Web,2017-01-23,Closed with explanation,Yes,No,2307001,Student loan Federal student loan servicing Ca...,"[student, loan, federal, student, loan, servic...","[0.15580810173414647, 0.17453869070159272, 0.1..."
